In [18]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from catboost import CatBoostClassifier

# Pré-processamento #

### Base e encoder manual ###

In [2]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df_encod_manual = pd.DataFrame.copy(df)

In [5]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [6]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [7]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [8]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [9]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [10]:
previsores_label = df.iloc[:, 0:11].values

In [11]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [12]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [13]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [14]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [15]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [16]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [17]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)

# Previsores #
<span style="font-size: 13px;">
<li> <strong>previsores</strong> -> Atributos codificados manualmente sem escalonamento.</li></br>
<li> <strong>previsoresHot_esc</strong> -> Atributos codificados com LabelEncoder e OneHotEncoder e escalonados.</li></br>
<li> <strong>previsores_esc</strong> -> Atributos codificados manualmente e escalonados.</li></br>
<li> <strong>previsores_hot</strong> -> Atributos codificados com OneHotEncoder sem escalonamento.</li></br>
<li> <strong>previsores_label</strong> -> Atributos codificados com LabelEncoder e sem escalonamento. </li></br>
</span>

# CatBoost #
<span style="font-size: 13px;">

**Conceitos Fundamentais** </br>

O CatBoost é baseado na técnica de Gradient Boosting, que combina vários modelos de aprendizado de máquina fracos para criar um modelo mais robusto. Ele também se destaca por sua capacidade de lidar diretamente com variáveis categóricas, sem a necessidade de codificação prévia.

**Gradient Boosting:** </br>

O Gradient Boosting é uma técnica de aprendizado de máquina que cria um modelo preditivo combinando várias iterações de modelos simples. Cada modelo é ajustado para corrigir os erros do modelo anterior.

**Variáveis Categóricas:** </br>

As variáveis categóricas são características que representam uma categoria ou grupo, em vez de um valor numérico. Exemplos incluem cores, países e tipos de produtos. Tradicionalmente, essas variáveis precisam ser codificadas antes de serem usadas em modelos de aprendizado de máquina.

**Funcionamento** </br>

O CatBoost se destaca por sua capacidade de lidar diretamente com variáveis categóricas, sem a necessidade de codificação prévia. Ele usa uma técnica chamada Codificação de Impacto para transformar variáveis categóricas em valores numéricos durante o treinamento.

**Codificação de Impacto:** </br>

A Codificação de Impacto é uma técnica que atribui a cada categoria de uma variável categórica um valor com base em seu impacto nas previsões do modelo. Isso permite que o modelo aprenda a importância de cada categoria diretamente dos dados.

**Parâmetros** </br>

O CatBoost oferece uma variedade de parâmetros que podem ser ajustados para otimizar o desempenho do modelo. Aqui estão alguns exemplos:

**Parâmetros de Regularização:** </br>

- `depth`: A profundidade máxima de cada árvore.
- `learning_rate`: Taxa de aprendizado para atualizar os modelos.
- `l2_leaf_reg`: Força da regularização L2.

**Parâmetros de Treinamento:** </br>

- `iterations`: O número máximo de iterações de treinamento.
- `early_stopping_rounds`: Número de iterações sem melhoria antes de parar o treinamento.
</span>

In [19]:
previsores_cat = df.iloc[:, 0:11]

In [20]:
alvo_cat = df.iloc[:, 11]

In [21]:
x_train, x_test, y_train, y_test = train_test_split(previsores_cat, alvo_cat, test_size = 0.3, random_state = 3)

In [22]:
categoricas = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [23]:
catboost = CatBoostClassifier(
    task_type = 'CPU', iterations = 120,
    learning_rate = 0.1, depth = 8,
    random_state = 5, eval_metric = 'Accuracy'
)

In [24]:
catboost.fit(x_train, y_train, cat_features = categoricas, plot = True, eval_set = (x_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8861154	test: 0.8188406	best: 0.8188406 (0)	total: 172ms	remaining: 20.5s
1:	learn: 0.8580343	test: 0.8333333	best: 0.8333333 (1)	total: 191ms	remaining: 11.3s
2:	learn: 0.8549142	test: 0.8333333	best: 0.8333333 (1)	total: 222ms	remaining: 8.65s
3:	learn: 0.8439938	test: 0.8442029	best: 0.8442029 (3)	total: 228ms	remaining: 6.6s
4:	learn: 0.8299532	test: 0.8369565	best: 0.8442029 (3)	total: 236ms	remaining: 5.42s
5:	learn: 0.8455538	test: 0.8369565	best: 0.8442029 (3)	total: 249ms	remaining: 4.73s
6:	learn: 0.8564743	test: 0.8369565	best: 0.8442029 (3)	total: 264ms	remaining: 4.26s
7:	learn: 0.8439938	test: 0.8369565	best: 0.8442029 (3)	total: 275ms	remaining: 3.85s
8:	learn: 0.8533541	test: 0.8442029	best: 0.8442029 (3)	total: 290ms	remaining: 3.58s
9:	learn: 0.8564743	test: 0.8405797	best: 0.8442029 (3)	total: 308ms	remaining: 3.38s
10:	learn: 0.8533541	test: 0.8442029	best: 0.8442029 (3)	total: 327ms	remaining: 3.23s
11:	learn: 0.8439938	test: 0.8442029	best: 0.8442029 (

In [25]:
previsoes = catboost.predict(x_test)